# Test for anaerobicity using different films in 96 Well plates #
### Notebook can be used to generate Fig. 4a and Supplementary Fig. S2 ###

### Import necessary modules ###

In [2]:
import pandas as pd
from scipy.stats import linregress, ttest_ind
import numpy as np
import sys
from plotly import tools, subplots, colors
import plotly.graph_objs as go
import pickle
import plotly.io as pio
pio.templates.default = "none"

if 'ipykernel' in sys.modules:
    from plotly.offline import init_notebook_mode
    from plotly.offline import iplot as plot
    from IPython.display import HTML
    HTML("""
         <script>
          var waitForPlotly = setInterval( function() {
          if( typeof(window.Plotly) !== "undefined" ){
          MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });
          MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);
          clearInterval(waitForPlotly);}}, 250 );
        </script>
        """
    )
    init_notebook_mode(connected=True)


### Load file with processed biomass yield (ratio of final to initial biomass) data ###

In [3]:
file = pd.ExcelFile('film_test_processed_data.xlsx')
df_dict = {}

for sheet in file.sheet_names:
    df_dict[sheet] = file.parse(sheet, index_col=0)
    
colorscale = [[0, '#2164AF'],[0.5, '#EAEAEA'], [1, '#D9482A']]

### Make violin plots with cell yield data for each well ###

In [6]:
fig = go.Figure()
trace_list = []

for film in list(reversed(list(df_dict.keys()))):
    if film not in ['breathe_easy_aerobic']:
        trace_list.append(go.Violin(x=df_dict[film].values.flatten(),box_visible=True,
                                meanline_visible=False, name=film.title(), opacity=1,
                                line_color='black', fillcolor='#B4DCB4',
                                marker_color='black', marker_size=5,line_width=0.45, 
                                showlegend=False))

    
    
layout = go.Layout(height=450, width=425, showlegend=False,
                   
                   xaxis=dict(title='Biomass Yield (Ratio of Final Biomass to Initial Biomass)',showline=True, linewidth=1, linecolor='black', mirror=True, 
                              ticks='outside', ticklen=4, tickangle=0, tickcolor='black',
                              tickfont=dict(size=13, family='Arial', color='black'),
                              side='bottom', showgrid=True,nticks=10,range=(16,28)),
                           
                   yaxis=dict(
                              title_standoff=0, titlefont=dict(family='Arial', size=16, color='black'),
                              showline=True, linewidth=1, linecolor='black',mirror=False,
                              ticks='outside', ticklen=4, tickangle=0, tickcolor='black', 
                              tickfont=dict(family='Arial',size=13, color='black'),
                              showgrid=True))


fig = go.Figure(data=trace_list,layout=layout)
plot(fig)
#pio.write_image(fig,"Figures/fig_3_film_violin.svg",format='svg')


### Make heatmaps cell yield data for each well ###

In [7]:
for film in df_dict:
    if film not in ['breathe_easy_aerobic']:
        trace_list = []
        trace_list.append(go.Heatmap(
                           z=list(reversed(df_dict[film].values - np.nanmedian(df_dict[film].values.flatten()))),
                           x=df_dict[film].columns,
                           y=list(reversed(df_dict[film].index)),
                           hoverongaps = False, zmin=-1.5, zmax=1.5,colorscale=colorscale,
                            showscale=True,
                           colorbar=dict(dtick=0.5,ticks='outside',tickcolor='black',outlinecolor='black',
                                         outlinewidth=0.25,thickness=15,len=1.5)))
        
        layout = go.Layout(height=310, width=355, showlegend=False, title_text=film,
                   
                   xaxis=dict(showline=True, linewidth=1, linecolor='black', mirror=True, 
                              ticks='', ticklen=4, tickangle=0, tickcolor='black',
                              tickfont=dict(size=13, family='Arial', color='black'),
                              side='bottom', showgrid=False),
                           
                   yaxis=dict(
                              title_standoff=0, titlefont=dict(family='Arial', size=16, color='black'),
                              showline=True, linewidth=1, linecolor='black',mirror=True,
                              ticks='', ticklen=4, tickangle=0, tickcolor='black', nticks=0,
                              tickfont=dict(family='Arial',size=13, color='black'),
                              showgrid=False))
        
        fig = go.Figure(data=trace_list, layout=layout)
        fig.show()
        #pio.write_image(fig,"Figures/fig_3_variation_"+film+".svg",format='svg')


In [8]:
for film in df_dict:
    if film not in ['breathe_easy_aerobic']:
        trace_list = []
        annotations = go.Annotations()
        
        trace_list.append(go.Heatmap(
                           z=list(reversed(df_dict[film].values - np.nanmedian(df_dict[film].values.flatten()))),
                           x=df_dict[film].columns,
                           y=list(reversed(df_dict[film].index)), 
                           hoverongaps = False, zmin=-1.5, zmax=1.5,colorscale=colorscale,
                            showscale=True,
                           colorbar=dict(dtick=0.5,ticks='outside',tickcolor='black',outlinecolor='black',
                                         outlinewidth=0.25,thickness=15,len=1)))
        for n, row in enumerate(df_dict[film].values):
            for m,val in enumerate(row):
                annotations.append(go.layout.Annotation(text=str(df_dict[film].values[n][m]),
                                                 x=df_dict[film].columns[m],
                                                 y=list((df_dict[film].index))[n],
                                                 xref='x1',yref='y1',showarrow=False))
        layout = go.Layout(height=450, width=575, showlegend=False, title_text=film,annotations=annotations,
                   
                   xaxis=dict(showline=True, linewidth=1, linecolor='black', mirror=True, 
                              ticks='', ticklen=4, tickangle=0, tickcolor='black', nticks=20,
                              tickfont=dict(size=13, family='Arial', color='black'),
                              side='top', showgrid=False),
                           
                   yaxis=dict(
                              title_standoff=0, titlefont=dict(family='Arial', size=16, color='black'),
                              showline=True, linewidth=1, linecolor='black',mirror=True,
                              ticks='', ticklen=4, tickangle=0, tickcolor='black', nticks=0,
                              tickfont=dict(family='Arial',size=13, color='black'),
                              showgrid=False))
        
        fig = go.Figure(data=trace_list, layout=layout)
        fig.show()
        #pio.write_image(fig,"Figures/si_fig_3_variation_"+film+".svg",format='svg')


C:\Users\User\anaconda3\envs\default\lib\site-packages\plotly\graph_objs\_deprecations.py:68: DeprecationWarning:

plotly.graph_objs.Annotations is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.layout.Annotation
  - plotly.graph_objs.layout.scene.Annotation


